In [ ]:
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Azure.AI.OpenAI,*-*"
#r "nuget: Azure.Core"

using Azure.AI.OpenAI;
using Azure;

In [ ]:
#!import config/Config.cs

In [ ]:
Console.WriteLine($"UseAzureOpenAI? {Config.UseAzureOpenAI}");


In [ ]:
var client = Config.UseAzureOpenAI
    ? new OpenAIClient(
        new Uri("https://your-azure-openai-resource.com/"),
        new AzureKeyCredential("your-azure-openai-resource-api-key"))
    : new OpenAIClient(Config.ApiKey);

In [ ]:
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    DeploymentName = Config.ModelName, // Use DeploymentName for "model" with non-Azure clients
    Messages =
    {
        // The system message represents instructions or other guidance about how the assistant should behave
        new ChatRequestSystemMessage("You are a helpful assistant. You will talk like a pirate."),
        // User messages represent current or historical input from the end user
        new ChatRequestUserMessage("Can you help me?"),
        // Assistant messages represent historical responses from the assistant
        new ChatRequestAssistantMessage("Arrrr! Of course, me hearty! What can I do for ye?"),
        new ChatRequestUserMessage("What's the best way to train a parrot?"),
    }
};

Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);
ChatResponseMessage responseMessage = response.Value.Choices[0].Message;
Console.WriteLine($"[{responseMessage.Role.ToString().ToUpperInvariant()}]: {responseMessage.Content}");

Add the response to the chat completions history...

In [ ]:
chatCompletionsOptions.Messages.Add(new ChatRequestAssistantMessage(responseMessage.Content));
chatCompletionsOptions.Messages

## Prompt Engineering

You can combine those steps into a simple RAG function that:

1. intializes chat history with a prompt
2. accepts user input
3. adds the input to chat history
4. passes the entire history to the model for a new completion
5. returns the result from the LLM
6. repeats from step 2

See `BasicChat.cs` for the implementation, and try it out below.

In [ ]:
#!import rag/BasicChat.cs

In [ ]:
var chat = new BasicChat(client, Config.ModelName, "You are a helpful assistant. You will talk like Yoda.");

In [ ]:
chat.Say("Build me a ship!")

In [ ]:
chat.Say("What did I just ask you about?")

In [ ]:
chat.Say("What is a marble?")

In [ ]:
chat.Say("How many marbles are on the boat?")

In [ ]:
chat.Say("Place 100 marbles on the tall ship, and 3 marbles on the rowboat. How many marbles will there be in total?")

In [ ]:
chat.Say("How many marbles are on the boat?")

In [ ]:
chat.Say("How many marbles are on the boat? If you're not sure which boat, ask a clarifying question.")

In [ ]:
chat.Say("How many marbles are on the sailing ship?")

In [ ]:
chat.Say("List the boats, ordered by the number of marbles on each from most to least.")